In [2]:
%load_ext tikzmagic
%matplotlib inline

# 성능과 변수선택

이 절에서는 모형의 성능을 가늠하는 방법과 성능 향상을 위해 어떤 변수를 선택해야 하는지를 설명한다. 예제 데이터로는 상관관계를 설명할 때 사용하였던 보스턴 집값 데이터를 사용한다. 보스턴 집값 데이터는 13개의 독립변수를 이용하여 medv라는 1개의 종속변수를 예측한다.

In [4]:
import statsmodels.api as sm

boston = sm.datasets.get_rdataset("Boston", "MASS").data
boston

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


일단 범죄율 crim 변수만 사용하여 medv 값을 예측해 보자. 이 때 모형 문자열은 다음과 같다.

```
"medv ~ scale(crim)"
```

선형회귀분석 결과는 다음과 같다.

In [5]:
import statsmodels.api as sm

formula1 = "medv ~ scale(crim)"
result1 = sm.OLS.from_formula(formula1, boston).fit()
print(result1.summary())

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.151
Model:                            OLS   Adj. R-squared:                  0.149
Method:                 Least Squares   F-statistic:                     89.49
Date:                Tue, 26 Jul 2022   Prob (F-statistic):           1.17e-19
Time:                        10:00:56   Log-Likelihood:                -1798.9
No. Observations:                 506   AIC:                             3602.
Df Residuals:                     504   BIC:                             3610.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      22.5328      0.377     59.745      

## 결정계수

분석결과요약에서 상단 오른쪽 열의 가장 위에 있는 `R-squared` 수치는 <b>결정계수(Coefficient of Determination)</b>라고 하는 값으로 선형회귀분석 성능을 표시한다. 결정계수는 수학기호로 $R^2$라고 표시하므로 `R-squared`라는 이름을 사용하였다.

최소자승법은 원 데이터의 종속변수 $y$ 값과 예측된 종속변수 $\hat{y}$의 차이인 잔차제곱합 RSS(Residual Sum of Squares) 수치를 가장 작게 만드는 방법이라고 설명하였다. 하지만 RSS 값 자체는 모든 데이터에 대해 예측 오차를 제곱한 수치를 더한 것이므로 데이터의 수가 증가하면 같이 증가한다. 따라서 같은 분석을 하더라도 데이터의 개수가 적은 경우와 데이터의 개수가 많은 경우를 직접 비교할 수 없다. 이러한 단점을 극복하기 위해 잔차제곱합 RSS를 특정한 범위내로 정규화(normalization)시킨 것이 결정계수다. 결정계수는 다음과 같이 계산한다.

$$
R^2 = 1 - \dfrac{\text{RSS}}{\text{TSS}}
$$

또는

$$
R^2 = \dfrac{\text{ESS}}{\text{TSS}}
$$

두 식 중 어떤 것을 사용해도 같은 값이 나온다. 위 식에서 TSS와 ESS는 각각 다음과 같이 정의한다.

$$
\text{TSS} = \sum_{i=1}^N (y_i-\bar{y})^2
$$

$$
\text{ESS}=\sum_{i=1}^N (\hat{y}_i -\bar{\hat{y}})^2 
$$

## F-검정

개별 개수가 아닌 전체 회귀 계수가 모두 의미가 있는지 확인하는 경우에는 다음과 같은 귀무 가설을 생각할 수 있다.

$$
H_0:w_0=w_1=\cdots=w_K−1=0
$$

이는 전체 독립 변수 중 어느 것도 의미를 가진 것이 없다는 뜻이다. 대부분의 경우, 이 귀무가설은 기각된다. 다만 유의 확률이 얼마나 작은가에 따라서 기각되는 정도가 달라진다. 유의 확률이 작으면 작을수록 더 강력하게 기각된 것이므로 더 의미가 있는 모형이라고 할 수 있다. 따라서 여러 모형의 유의 확률을 비교하여 어느 모형이 더 성능이 좋은가를 비교할 때 이 유의 확률을 사용한다. 이러한 검정을 <b>선형회귀분석 F-검정(regression F-test)</b>이라고 한다.

분석결과요약에서 F-statistic라고 표시된 400.3이라는 값이 회귀분석 F-검정의 검정통계량이고 Prob (F-statistic)로 표시된 2.21e-36라는 값이 유의확률이다.